# Severe Weather Capstone - Data Collection & Wrangling

Greg Welliver   

In [1]:
# Import relevant libraries and packages.
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns 
import statsmodels.api as sm

from statsmodels.graphics.api import abline_plot
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn import linear_model, preprocessing 
import warnings
from scipy import stats
import re
from glob import glob, iglob
from datetime import datetime
import string


In [2]:
#NotebookApp.iopub_data_rate_limit=1.0e10
#https://stackoverflow.com/questions/43288550/iopub-data-rate-exceeded-in-jupyter-notebook-when-viewing-image

file location for downloads: 
    https://www.ncei.noaa.gov/pub/data/swdi/stormevents/csvfiles/

## Data Collection

- storm files were collected from the Iowa Environmental Mesonet: https://mesonet.agron.iastate.edu/nws/

### working code, make markdown for now
### All annual storm data files are saved on my local machine.  This code gathers all of the files and combines them into one file.
filenames = glob('../Data/*.csv')
print("There is a total of {} files.".format(len(filenames)))

target_path = '../Data/all_storm_data.csv'

try:
    # Read in Summary File is exists
    all_storm_data = pd.read_csv(target_path)
except:
    # Read in all Subfiles
    storm_data = [pd.read_csv(filepath) for filepath in filenames]
    all_storm_data = pd.concat(storm_data)
    
    # Create Summary File for faster processing
    hot100_all.to_csv(target_path,index=False)

print("The total number of observations is {}.".format(len(all_storm_data)))
all_storm_data.head()

In [3]:
# load data
#df = pd.read_csv("../Data/StormEvents_details-ftp_v1.0_d2001_c20220425.csv")
#df = pd.read_parquet("../Data/all_storm_data.pqt")
#df = pd.read_csv("../Data/all_storm_data4.csv")

In [4]:
#df.head()

In [5]:
#df.shape

In [6]:
#df.isna().sum()

In [7]:
#df.info()

for row in df["STATE_FIPS"][:10]:
    res = row.split(".", 1)[0]
    print(res)

# WORKING, MARKDOWN UNTIL FINAL
# drop unnecessary columns
df.drop(['CATEGORY', 'DATA_SOURCE', 'BEGIN_RANGE', 'BEGIN_AZIMUTH', 'BEGIN_LOCATION', 'END_RANGE', 'END_AZIMUTH', 'END_LOCATION', 'TOR_OTHER_WFO', 'TOR_OTHER_CZ_STATE', 'TOR_OTHER_CZ_FIPS', 'TOR_OTHER_CZ_NAME', 'CZ_TIMEZONE', 'WFO', 'CZ_TYPE', 'DAMAGE_CROPS', 'CZ_NAME', 'SOURCE', 'BEGIN_DAY', 'END_YEARMONTH', 'END_DAY', 'END_TIME', 'EPISODE_ID', 'EVENT_ID', 'STATE_FIPS', 'CZ_FIPS', 'END_DATE_TIME'], axis=1, inplace=True)

# WORKING, MARKDOWN UNTIL FINAL
df['STATE_FIPS'] = df['STATE_FIPS'].astype(object)

# WORKING, MARKDOWN UNTIL FINAL
# Columns to replace nulls with NA:
cols_na = ['EVENT_NARRATIVE', 'EPISODE_NARRATIVE', 'BEGIN_LAT', 'BEGIN_LON', 'END_LAT', 'END_LON', 'TOR_F_SCALE', 'MAGNITUDE_TYPE', 'FLOOD_CAUSE', 'STATE', 'STATE_FIPS']

for x in cols_na:
    print(df[x].isna().sum())

# WORKING, MARKDOWN UNTIL FINAL
for x in cols_na:
    df[x] = df[x].fillna('NA')

for x in cols_na:
    print(df[x].isna().sum())

# WORKING, MARKDOWN UNTIL FINAL
# Columns to replace nulls with 0:

cols_0 = ['MAGNITUDE', 'TOR_LENGTH', 'TOR_WIDTH', 'DAMAGE_PROPERTY', 'INJURIES_DIRECT', 'INJURIES_INDIRECT', 'DEATHS_DIRECT', 'DEATHS_INDIRECT']


for x in cols_0:
    print(df[x].isna().sum())

# WORKING, MARKDOWN UNTIL FINAL
for x in cols_0:
    df[x] = df[x].fillna(0)

for x in cols_0:
    print(df[x].isna().sum())

In [8]:
#df.isna().sum()

In [9]:
#df.dtypes

# WORKING, MARKDOWN UNTIL FINAL
# remove NA values from state FIPS
df = df[df['STATE_FIPS'] != "NA"].reset_index()

# convert STATE_FIPS to INT so can use it for lookup later
for x in df['STATE_FIPS']:
    x = int(x)

df['STATE_FIPS'] = df['STATE_FIPS'].astype(int)


df['STATE_FIPS'] = df['STATE_FIPS'].astype(object)

df['STATE_FIPS']

df['CZ_FIPS'] = df['CZ_FIPS'].astype(str)

df['CZ_FIPS']

df['CZ_FIPS'][:5]

In [10]:
#df[249750:249760]

df['CZ_FIPS'][249750:249760]

# WORKING, MARKDOWN UNTIL FINAL
# add "0" or "00" to CZ FIPS so that it can be used to match later
for i in (range(len(df['CZ_FIPS']))):
    if len(df['CZ_FIPS'][i]) == 2:
#        df['CZ_FIPS'][i] = df['CZ_FIPS'][i].astype(str)
        df['CZ_FIPS'][i] = "0" + df['CZ_FIPS'][i]
#        print(df['CZ_FIPS'][i])
    elif len(df['CZ_FIPS'][i]) == 1:
        df['CZ_FIPS'][i] = "00" + df['CZ_FIPS'][i]
#        print(df['CZ_FIPS'][i])
#     else:
#         row

df['STATE_FIPS'] = df['STATE_FIPS'].astype(str)

# WORKING, MARKDOWN UNTIL FINAL
# add "0" to state FIPS so that it can be used to match later
for i in (range(len(df['STATE_FIPS']))):
    if len(df['STATE_FIPS'][i]) == 1:
#        df['CZ_FIPS'][i] = df['CZ_FIPS'][i].astype(str)
        df['STATE_FIPS'][i] = "0" + df['STATE_FIPS'][i]
#        print(df['CZ_FIPS'][i])

#df["CZ_FIPS"] = df.apply(lambda x: "0" + x if len(x) == 2)

# WORKING, MARKDOWN UNTIL FINAL
# concatenate STATE FIPS and CZ FIPS into one column so that it can be used to match
df['ST_CT_FIPS'] = df['STATE_FIPS'].astype(str) + df['CZ_FIPS'].astype(str)

# WORKING, MARKDOWN UNTIL FINAL
# remove all of the K's, M's, and B's in the DAMAGE_PROPERTY column and multiply them by appropriate values
d = {r"(\d)K$": r"\1*1000", r"M$": r"*1000000", r"B$": r"*1000000000", r"^K$": r"1000"}

#r stands for raw string
#dollar is end of the line

# for every key and value, run this code
for k,v in d.items():
     df["DAMAGE_PROPERTY"] = df["DAMAGE_PROPERTY"].str.replace(k, v, regex=True).fillna("0.0")
#df["DAMAGE_PROPERTY"].apply(eval)
df["DAMAGE_PROPERTY"] = df["DAMAGE_PROPERTY"].apply(eval)

# WORKING, MARKDOWN UNTIL FINAL
# convert date strings to datetimes
df['BEGIN_DATE_TIME'] =  pd.to_datetime(df['BEGIN_DATE_TIME'])
df['END_DATE_TIME'] =  pd.to_datetime(df['END_DATE_TIME'])

# WORKING, MARKDOWN UNTIL FINAL
# calculate duration of storm
df['DURATION'] = df['END_DATE_TIME'] - df['BEGIN_DATE_TIME']

# convert storm duration to minutes
for i in (range(len(df['DURATION']))):
    df['DURATION'][i] = df['DURATION'][i].total_seconds() / 60

# WORKING, MARKDOWN UNTIL FINAL
# code to calculate coverage area of the storm

# calculate  beginning and end latitude difference
df['LAT_DIFF'] = (df['END_LAT'] - df['BEGIN_LAT']).abs()

# calculate  beginning and end longitude difference
df['LON_DIFF'] = (df['END_LON'] - df['END_LON']).abs()

# combine two columns to calculate total size of storm
df['STORM_AREA'] = df['LON_DIFF'] + df['LAT_DIFF']

# since we don't need the difference columns anymore, drop those. also END LAT and LON columsn, since don't need those either
df.drop(['LAT_DIFF', 'LON_DIFF', 'END_LON', 'END_LAT'], axis=1, inplace=True)

## Part 2 start here

In [11]:
#load the data
# df = pd.read_csv("../Data/all_storm_data4.csv", index_col=[0])
# df.drop(['index'], axis=1, inplace=True)
df = pd.read_parquet("../Data/all_storm_data7.pqt")

In [12]:
df.head()

,BEGIN_YEARMONTH,BEGIN_TIME,STATE,YEAR,MONTH_NAME,EVENT_TYPE,BEGIN_DATE_TIME,INJURIES_DIRECT,INJURIES_INDIRECT,DEATHS_DIRECT,...,TOR_F_SCALE,TOR_LENGTH,TOR_WIDTH,BEGIN_LAT,BEGIN_LON,EPISODE_NARRATIVE,EVENT_NARRATIVE,ST_CT_FIPS,DURATION,STORM_AREA
0,202202,2118,NEVADA,2022,February,High Wind,2022-02-20 21:18:00,0,0,0,...,None,0.0,0.0,NaN,NaN,Strong winds increased ahead of an approaching...,"Station (UP994) 3.1 SE West Wendover, Elevatio...",32033,60.0,NaN
1,202202,800,NEVADA,2022,February,Heavy Snow,2022-02-21 08:00:00,0,0,0,...,None,0.0,0.0,NaN,NaN,A low centered over northern and central Nevad...,Thirteen inches fell at station (BCSN2) Big Cr...,32037,1560.0,NaN
2,202202,200,NEVADA,2022,February,Heavy Snow,2022-02-22 02:00:00,0,0,0,...,None,0.0,0.0,NaN,NaN,A low centered over northern and central Nevad...,Fifteen inches fell at station (TJMN2) Toe Jam...,32031,420.0,NaN
3,202202,1609,ATLANTIC SOUTH,2022,February,Waterspout,2022-02-18 16:09:00,0,0,0,...,None,0.0,0.0,30.050,-81.1700,Pre-frontal showers and thunderstorms moved so...,A brief waterspout was observed offshore of So...,87452,0.0,0.0000
4,202202,0,AMERICAN SAMOA,2022,February,Heavy Rain,2022-02-02 00:00:00,0,0,0,...,None,0.0,0.0,-14.333,-170.7157,A surface trough over the Islands held the po...,"Over a 24-hour period, WSO Pago Pago recorded ...",97002,1440.0,0.0063


In [13]:
df.head(20).T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
BEGIN_YEARMONTH,202202,202202,202202,202202,202202,202202,202202,202202,202202,202202,202202,202202,202202,202202,202202,202202,202202,202202,202202,202202
BEGIN_TIME,2118,800,200,1609,0,500,100,100,200,400,400,300,500,2026,900,900,900,900,900,900
STATE,NEVADA,NEVADA,NEVADA,ATLANTIC SOUTH,AMERICAN SAMOA,AMERICAN SAMOA,KANSAS,KANSAS,MASSACHUSETTS,MASSACHUSETTS,MASSACHUSETTS,RHODE ISLAND,FLORIDA,MONTANA,NEW YORK,NEW YORK,NEW YORK,NEW YORK,NEW YORK,NEW YORK
YEAR,2022,2022,2022,2022,2022,2022,2022,2022,2022,2022,2022,2022,2022,2022,2022,2022,2022,2022,2022,2022
MONTH_NAME,February,February,February,February,February,February,February,February,February,February,February,February,February,February,February,February,February,February,February,February
EVENT_TYPE,High Wind,Heavy Snow,Heavy Snow,Waterspout,Heavy Rain,Heavy Rain,Winter Storm,Winter Storm,Heavy Snow,Heavy Snow,Heavy Snow,Heavy Snow,Dense Fog,Winter Storm,Winter Storm,Winter Storm,Winter Storm,Winter Storm,Winter Storm,Winter Storm
BEGIN_DATE_TIME,2022-02-20 21:18:00,2022-02-21 08:00:00,2022-02-22 02:00:00,2022-02-18 16:09:00,2022-02-02 00:00:00,2022-02-12 05:00:00,2022-02-01 01:00:00,2022-02-01 01:00:00,2022-02-13 02:00:00,2022-02-13 04:00:00,2022-02-13 04:00:00,2022-02-13 03:00:00,2022-02-25 05:00:00,2022-02-01 20:26:00,2022-02-03 09:00:00,2022-02-03 09:00:00,2022-02-03 09:00:00,2022-02-03 09:00:00,2022-02-03 09:00:00,2022-02-03 09:00:00
INJURIES_DIRECT,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
INJURIES_INDIRECT,0,0,0,0,0,0,0,0,0,0,0,0,30,0,0,0,0,0,0,0
DEATHS_DIRECT,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [14]:
df['ST_CT_FIPS'] = df['ST_CT_FIPS'].astype(str)
df['ST_CT_FIPS'] = df['ST_CT_FIPS'].str.zfill(5)
df.ST_CT_FIPS

0          32033
1          32037
2          32031
3          87452
4          97002
           ...  
1555642    99001
1555643    05109
1555644    99001
1555645    99004
1555646    99005
Name: ST_CT_FIPS, Length: 1555647, dtype: object

In [15]:
df.head()

,BEGIN_YEARMONTH,BEGIN_TIME,STATE,YEAR,MONTH_NAME,EVENT_TYPE,BEGIN_DATE_TIME,INJURIES_DIRECT,INJURIES_INDIRECT,DEATHS_DIRECT,...,TOR_F_SCALE,TOR_LENGTH,TOR_WIDTH,BEGIN_LAT,BEGIN_LON,EPISODE_NARRATIVE,EVENT_NARRATIVE,ST_CT_FIPS,DURATION,STORM_AREA
0,202202,2118,NEVADA,2022,February,High Wind,2022-02-20 21:18:00,0,0,0,...,None,0.0,0.0,NaN,NaN,Strong winds increased ahead of an approaching...,"Station (UP994) 3.1 SE West Wendover, Elevatio...",32033,60.0,NaN
1,202202,800,NEVADA,2022,February,Heavy Snow,2022-02-21 08:00:00,0,0,0,...,None,0.0,0.0,NaN,NaN,A low centered over northern and central Nevad...,Thirteen inches fell at station (BCSN2) Big Cr...,32037,1560.0,NaN
2,202202,200,NEVADA,2022,February,Heavy Snow,2022-02-22 02:00:00,0,0,0,...,None,0.0,0.0,NaN,NaN,A low centered over northern and central Nevad...,Fifteen inches fell at station (TJMN2) Toe Jam...,32031,420.0,NaN
3,202202,1609,ATLANTIC SOUTH,2022,February,Waterspout,2022-02-18 16:09:00,0,0,0,...,None,0.0,0.0,30.050,-81.1700,Pre-frontal showers and thunderstorms moved so...,A brief waterspout was observed offshore of So...,87452,0.0,0.0000
4,202202,0,AMERICAN SAMOA,2022,February,Heavy Rain,2022-02-02 00:00:00,0,0,0,...,None,0.0,0.0,-14.333,-170.7157,A surface trough over the Islands held the po...,"Over a 24-hour period, WSO Pago Pago recorded ...",97002,1440.0,0.0063


In [16]:
df.head().T

,0,1,2,3,4
BEGIN_YEARMONTH,202202,202202,202202,202202,202202
BEGIN_TIME,2118,800,200,1609,0
STATE,NEVADA,NEVADA,NEVADA,ATLANTIC SOUTH,AMERICAN SAMOA
YEAR,2022,2022,2022,2022,2022
MONTH_NAME,February,February,February,February,February
EVENT_TYPE,High Wind,Heavy Snow,Heavy Snow,Waterspout,Heavy Rain
BEGIN_DATE_TIME,2022-02-20 21:18:00,2022-02-21 08:00:00,2022-02-22 02:00:00,2022-02-18 16:09:00,2022-02-02 00:00:00
INJURIES_DIRECT,0,0,0,0,0
INJURIES_INDIRECT,0,0,0,0,0
DEATHS_DIRECT,0,0,0,0,0


In [17]:
df.ST_CT_FIPS.sort_values(ascending=False)

402724     99153
1004930    99153
1217216    99153
1117745    99153
894782     99153
           ...  
476623     01001
214390     01001
84059      01001
493378     01001
393601     01001
Name: ST_CT_FIPS, Length: 1555647, dtype: object

## Combine Population Density, Home Price data


In [18]:
#load the data
#PopDen = pd.read_csv("../Data/Average_Household_Size_and_Population_Density_-_County_merge.csv", index_col=[0])
PopDen = pd.read_csv("../Data/Average_Household_Size_and_Population_Density_-_County_merge.csv")
#HomePrice = pd.read_csv("../Data/HPI_AT_BDL_county_merge.csv", index_col=[0])
#HomePrice = pd.read_csv("../Data/HPI_AT_BDL_county_merge.csv")
HomePrice = pd.read_excel("../Data/HPI_AT_BDL_county_merge.xlsx")

In [19]:
PopDen = PopDen[PopDen['FIPS_CODE'].notnull()]
PopDen = PopDen.reset_index(drop=True)

In [20]:
# code to fix FIPS_CODE column in PopDen

# convert to int to get rid of decimals
PopDen['FIPS_CODE'] = PopDen['FIPS_CODE'].astype(int)

# pad additional zeroes
PopDen['FIPS_CODE'] = PopDen['FIPS_CODE'].astype(str)
PopDen['FIPS_CODE'] = PopDen['FIPS_CODE'].str.zfill(5)

# code to fix FIPS CODE column in HomePrice

# pad additional zeroes
HomePrice['FIPS code'] = HomePrice['FIPS code'].astype(str)
HomePrice['FIPS code'] = HomePrice['FIPS code'].str.zfill(5)

In [21]:
PopDen.head(20).T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
OBJECTID,1.0,2.0,3.0,4.0,5.0,7.0,8.0,9.0,10.0,11.0,12.0,13.0,14.0,15.0,16.0,17.0,18.0,19.0,20.0,21.0
COUNTYNS,161526.0,161527.0,161528.0,161529.0,161530.0,161532.0,161533.0,161534.0,161535.0,161536.0,161537.0,161538.0,161539.0,161540.0,161541.0,161542.0,161543.0,161544.0,161545.0,161546.0
GEOID,1001.0,1003.0,1005.0,1007.0,1009.0,1013.0,1015.0,1017.0,1019.0,1021.0,1023.0,1025.0,1027.0,1029.0,1031.0,1033.0,1035.0,1037.0,1039.0,1041.0
ALAND,1539602123.0,4117546676.0,2292144655.0,1612167481.0,1670103911.0,2012002530.0,1569189622.0,1545085607.0,1433623321.0,1794477075.0,2365869835.0,3207494111.0,1564251835.0,1450663940.0,1758565925.0,1535202321.0,2202007854.0,1685850815.0,2669129020.0,1576952799.0
AWATER,25706961.0,1133055836.0,50538698.0,9602089.0,15015423.0,2701198.0,16627597.0,16971701.0,120308339.0,20592144.0,19144469.0,36657891.0,5285207.0,2354896.0,3907189.0,79700277.0,6632046.0,39987271.0,34231390.0,5388561.0
NAME,Autauga County,Baldwin County,Barbour County,Bibb County,Blount County,Butler County,Calhoun County,Chambers County,Cherokee County,Chilton County,Choctaw County,Clarke County,Clay County,Cleburne County,Coffee County,Colbert County,Conecuh County,Coosa County,Covington County,Crenshaw County
State,Alabama,Alabama,Alabama,Alabama,Alabama,Alabama,Alabama,Alabama,Alabama,Alabama,Alabama,Alabama,Alabama,Alabama,Alabama,Alabama,Alabama,Alabama,Alabama,Alabama
Unnamed: 7,Autauga County Alabama,Baldwin County Alabama,Barbour County Alabama,Bibb County Alabama,Blount County Alabama,Butler County Alabama,Calhoun County Alabama,Chambers County Alabama,Cherokee County Alabama,Chilton County Alabama,Choctaw County Alabama,Clarke County Alabama,Clay County Alabama,Cleburne County Alabama,Coffee County Alabama,Colbert County Alabama,Conecuh County Alabama,Coosa County Alabama,Covington County Alabama,Crenshaw County Alabama
FIPS_CODE,01001,01003,01005,01007,01009,01013,01015,01017,01019,01021,01023,01025,01027,01029,01031,01033,01035,01037,01039,01041
B25010_001E,2.59,2.61,2.49,2.99,2.77,2.94,2.49,2.46,2.41,2.6,2.41,2.57,2.51,2.58,2.56,2.48,2.73,2.55,2.44,2.73


In [22]:
HomePrice.head(20).T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
State,AL,AL,AL,AL,AL,AL,AL,AL,AL,AL,AL,AL,AL,AL,AL,AL,AL,AL,AL,AL
County,Autauga,Autauga,Autauga,Autauga,Autauga,Autauga,Autauga,Autauga,Autauga,Autauga,Autauga,Autauga,Autauga,Autauga,Autauga,Autauga,Autauga,Autauga,Autauga,Autauga
FIPS code,01001,01001,01001,01001,01001,01001,01001,01001,01001,01001,01001,01001,01001,01001,01001,01001,01001,01001,01001,01001
Year,1986,1987,1988,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005
Annual Change (%),.,-2.22,3.8,4.33,-0.51,2.82,3.02,3.67,4.1,3.54,2.58,3.46,3.62,1.67,2.14,3.8,2.62,1.88,2.66,7.17
HPI,100,97.78,101.49,105.88,105.35,108.32,111.59,115.7,120.44,124.7,127.93,132.35,137.14,139.43,142.41,147.82,151.69,154.55,158.66,170.04
HPI with 1990 base,94.92,92.81,96.33,100.51,100,102.82,105.93,109.82,114.32,118.37,121.43,125.63,130.18,132.36,135.18,140.32,143.99,146.7,150.61,161.41
HPI with 2000 base,70.22,68.66,71.26,74.35,73.97,76.06,78.36,81.24,84.57,87.57,89.83,92.94,96.3,97.91,100,103.8,106.52,108.52,111.41,119.4


In [23]:
df[df['ST_CT_FIPS'] == "01001"]

,BEGIN_YEARMONTH,BEGIN_TIME,STATE,YEAR,MONTH_NAME,EVENT_TYPE,BEGIN_DATE_TIME,INJURIES_DIRECT,INJURIES_INDIRECT,DEATHS_DIRECT,...,TOR_F_SCALE,TOR_LENGTH,TOR_WIDTH,BEGIN_LAT,BEGIN_LON,EPISODE_NARRATIVE,EVENT_NARRATIVE,ST_CT_FIPS,DURATION,STORM_AREA
1874,202206,1500,ALABAMA,2022,June,Heat,2022-06-14 15:00:00,0,0,0,...,None,0.0,0.0,NaN,NaN,Heat index values reached 105-107 degrees in a...,The heat index reached 105 degrees for one to ...,01001,120.0,NaN
2486,202210,200,ALABAMA,2022,October,Frost/Freeze,2022-10-19 02:00:00,0,0,0,...,None,0.0,0.0,NaN,NaN,Temperatures dropped into the 26 to 32 degree ...,Temperatures dropped as low as 29 degrees.,01001,240.0,NaN
2558,202210,2300,ALABAMA,2022,October,Frost/Freeze,2022-10-19 23:00:00,0,0,0,...,None,0.0,0.0,NaN,NaN,Temperatures dropped into the 26 to 32 degree ...,The temperature dropped to as low as 27 degrees.,01001,480.0,NaN
2565,202210,500,ALABAMA,2022,October,Frost/Freeze,2022-10-18 05:00:00,0,0,0,...,None,0.0,0.0,NaN,NaN,Temperatures dropped into the 29 to 32 degree ...,Temperatures dropped to as low as 30 degrees d...,01001,120.0,NaN
6282,202206,1100,ALABAMA,2022,June,Heat,2022-06-16 11:00:00,0,0,0,...,None,0.0,0.0,NaN,NaN,Heat index values reached 105 degrees or highe...,The heat index reached and exceeded 105 degree...,01001,420.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1534818,200106,1330,ALABAMA,2001,June,Thunderstorm Wind,2001-06-04 13:30:00,0,0,0,...,None,0.0,0.0,32.70000,-86.48333,None,A few trees were knocked down near Marbury.,01001,0.0,0.00
1534819,200106,1410,ALABAMA,2001,June,Thunderstorm Wind,2001-06-04 14:10:00,0,0,0,...,None,0.0,0.0,32.51667,-86.63333,None,Several trees and power lines were blown down ...,01001,15.0,0.00
1536339,200106,1630,ALABAMA,2001,June,Thunderstorm Wind,2001-06-05 16:30:00,0,0,0,...,None,0.0,0.0,32.50000,-86.58333,None,Trees were blown down near Booth and in the Ol...,01001,0.0,0.00
1548495,200108,900,ALABAMA,2001,August,Heavy Rain,2001-08-06 09:00:00,0,0,0,...,None,0.0,0.0,NaN,NaN,The remnants of Tropical Storm Barry moved nor...,None,01001,720.0,NaN


In [24]:
# working
# merge the population density data to the main dataframe
df = df.merge(PopDen['B01001_calc_PopDensity'], how = 'left',
                left_on = 'ST_CT_FIPS', right_on = PopDen['FIPS_CODE'])
#TopCountries.index = TopCountries.index + 1

# using pd.concat
# merge the population density data to the main dataframe
df = pd.concat([df, PopDen[['B01001_calc_PopDensity', 'Population']]],
                  keys = ['ST_CT_FIPS', 'FIPS_CODE'])
#TopCountries.index = TopCountries.index + 1

In [25]:
df.head(30).T

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
BEGIN_YEARMONTH,202202,202202,202202,202202,202202,202202,202202,202202,202202,202202,...,202202,202202,202201,202201,202201,202201,202201,202201,202201,202201
BEGIN_TIME,2118,800,200,1609,0,500,100,100,200,400,...,1100,1100,1200,1200,1200,1200,1200,1200,1200,1200
STATE,NEVADA,NEVADA,NEVADA,ATLANTIC SOUTH,AMERICAN SAMOA,AMERICAN SAMOA,KANSAS,KANSAS,MASSACHUSETTS,MASSACHUSETTS,...,VERMONT,VERMONT,OHIO,OHIO,OHIO,OHIO,OHIO,OHIO,OHIO,OHIO
YEAR,2022,2022,2022,2022,2022,2022,2022,2022,2022,2022,...,2022,2022,2022,2022,2022,2022,2022,2022,2022,2022
MONTH_NAME,February,February,February,February,February,February,February,February,February,February,...,February,February,January,January,January,January,January,January,January,January
EVENT_TYPE,High Wind,Heavy Snow,Heavy Snow,Waterspout,Heavy Rain,Heavy Rain,Winter Storm,Winter Storm,Heavy Snow,Heavy Snow,...,Winter Storm,Winter Storm,Winter Weather,Winter Weather,Winter Weather,Winter Weather,Winter Weather,Winter Weather,Winter Weather,Winter Weather
BEGIN_DATE_TIME,2022-02-20 21:18:00,2022-02-21 08:00:00,2022-02-22 02:00:00,2022-02-18 16:09:00,2022-02-02 00:00:00,2022-02-12 05:00:00,2022-02-01 01:00:00,2022-02-01 01:00:00,2022-02-13 02:00:00,2022-02-13 04:00:00,...,2022-02-03 11:00:00,2022-02-03 11:00:00,2022-01-16 12:00:00,2022-01-16 12:00:00,2022-01-16 12:00:00,2022-01-16 12:00:00,2022-01-16 12:00:00,2022-01-16 12:00:00,2022-01-16 12:00:00,2022-01-16 12:00:00
INJURIES_DIRECT,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
INJURIES_INDIRECT,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
DEATHS_DIRECT,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [26]:
### FIXING BELOW

In [27]:
# merge the population data to the main dataframe
df = df.merge(PopDen['Population'], how = 'left',
                left_on = 'ST_CT_FIPS', right_on = PopDen['FIPS_CODE'])
#TopCountries.index = TopCountries.index + 1

In [28]:
df[df['ST_CT_FIPS'] == "01001"]

,BEGIN_YEARMONTH,BEGIN_TIME,STATE,YEAR,MONTH_NAME,EVENT_TYPE,BEGIN_DATE_TIME,INJURIES_DIRECT,INJURIES_INDIRECT,DEATHS_DIRECT,...,TOR_WIDTH,BEGIN_LAT,BEGIN_LON,EPISODE_NARRATIVE,EVENT_NARRATIVE,ST_CT_FIPS,DURATION,STORM_AREA,B01001_calc_PopDensity,Population
1874,202206,1500,ALABAMA,2022,June,Heat,2022-06-14 15:00:00,0,0,0,...,0.0,NaN,NaN,Heat index values reached 105-107 degrees in a...,The heat index reached 105 degrees for one to ...,01001,120.0,NaN,35.853419,58802.0
2486,202210,200,ALABAMA,2022,October,Frost/Freeze,2022-10-19 02:00:00,0,0,0,...,0.0,NaN,NaN,Temperatures dropped into the 26 to 32 degree ...,Temperatures dropped as low as 29 degrees.,01001,240.0,NaN,35.853419,58802.0
2558,202210,2300,ALABAMA,2022,October,Frost/Freeze,2022-10-19 23:00:00,0,0,0,...,0.0,NaN,NaN,Temperatures dropped into the 26 to 32 degree ...,The temperature dropped to as low as 27 degrees.,01001,480.0,NaN,35.853419,58802.0
2565,202210,500,ALABAMA,2022,October,Frost/Freeze,2022-10-18 05:00:00,0,0,0,...,0.0,NaN,NaN,Temperatures dropped into the 29 to 32 degree ...,Temperatures dropped to as low as 30 degrees d...,01001,120.0,NaN,35.853419,58802.0
6282,202206,1100,ALABAMA,2022,June,Heat,2022-06-16 11:00:00,0,0,0,...,0.0,NaN,NaN,Heat index values reached 105 degrees or highe...,The heat index reached and exceeded 105 degree...,01001,420.0,NaN,35.853419,58802.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1534818,200106,1330,ALABAMA,2001,June,Thunderstorm Wind,2001-06-04 13:30:00,0,0,0,...,0.0,32.70000,-86.48333,None,A few trees were knocked down near Marbury.,01001,0.0,0.00,35.853419,58802.0
1534819,200106,1410,ALABAMA,2001,June,Thunderstorm Wind,2001-06-04 14:10:00,0,0,0,...,0.0,32.51667,-86.63333,None,Several trees and power lines were blown down ...,01001,15.0,0.00,35.853419,58802.0
1536339,200106,1630,ALABAMA,2001,June,Thunderstorm Wind,2001-06-05 16:30:00,0,0,0,...,0.0,32.50000,-86.58333,None,Trees were blown down near Booth and in the Ol...,01001,0.0,0.00,35.853419,58802.0
1548495,200108,900,ALABAMA,2001,August,Heavy Rain,2001-08-06 09:00:00,0,0,0,...,0.0,NaN,NaN,The remnants of Tropical Storm Barry moved nor...,None,01001,720.0,NaN,35.853419,58802.0


In [29]:
#### FIXING ABOVE

In [30]:
# merge the home price index data to the main dataframe
df = pd.merge(df, HomePrice,  how='left', left_on=['ST_CT_FIPS','YEAR'], right_on = ['FIPS code','Year'])
df.drop(['HPI with 2000 base', 'HPI with 1990 base', 'Annual Change (%)', 'Year', 'FIPS code', 'County', 'State',], axis=1, inplace=True)


In [31]:
df[df['ST_CT_FIPS'] == "01001"]

,BEGIN_YEARMONTH,BEGIN_TIME,STATE,YEAR,MONTH_NAME,EVENT_TYPE,BEGIN_DATE_TIME,INJURIES_DIRECT,INJURIES_INDIRECT,DEATHS_DIRECT,...,BEGIN_LAT,BEGIN_LON,EPISODE_NARRATIVE,EVENT_NARRATIVE,ST_CT_FIPS,DURATION,STORM_AREA,B01001_calc_PopDensity,Population,HPI
1874,202206,1500,ALABAMA,2022,June,Heat,2022-06-14 15:00:00,0,0,0,...,NaN,NaN,Heat index values reached 105-107 degrees in a...,The heat index reached 105 degrees for one to ...,01001,120.0,NaN,35.853419,58802.0,238.01
2486,202210,200,ALABAMA,2022,October,Frost/Freeze,2022-10-19 02:00:00,0,0,0,...,NaN,NaN,Temperatures dropped into the 26 to 32 degree ...,Temperatures dropped as low as 29 degrees.,01001,240.0,NaN,35.853419,58802.0,238.01
2558,202210,2300,ALABAMA,2022,October,Frost/Freeze,2022-10-19 23:00:00,0,0,0,...,NaN,NaN,Temperatures dropped into the 26 to 32 degree ...,The temperature dropped to as low as 27 degrees.,01001,480.0,NaN,35.853419,58802.0,238.01
2565,202210,500,ALABAMA,2022,October,Frost/Freeze,2022-10-18 05:00:00,0,0,0,...,NaN,NaN,Temperatures dropped into the 29 to 32 degree ...,Temperatures dropped to as low as 30 degrees d...,01001,120.0,NaN,35.853419,58802.0,238.01
6282,202206,1100,ALABAMA,2022,June,Heat,2022-06-16 11:00:00,0,0,0,...,NaN,NaN,Heat index values reached 105 degrees or highe...,The heat index reached and exceeded 105 degree...,01001,420.0,NaN,35.853419,58802.0,238.01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1534818,200106,1330,ALABAMA,2001,June,Thunderstorm Wind,2001-06-04 13:30:00,0,0,0,...,32.70000,-86.48333,None,A few trees were knocked down near Marbury.,01001,0.0,0.00,35.853419,58802.0,147.82
1534819,200106,1410,ALABAMA,2001,June,Thunderstorm Wind,2001-06-04 14:10:00,0,0,0,...,32.51667,-86.63333,None,Several trees and power lines were blown down ...,01001,15.0,0.00,35.853419,58802.0,147.82
1536339,200106,1630,ALABAMA,2001,June,Thunderstorm Wind,2001-06-05 16:30:00,0,0,0,...,32.50000,-86.58333,None,Trees were blown down near Booth and in the Ol...,01001,0.0,0.00,35.853419,58802.0,147.82
1548495,200108,900,ALABAMA,2001,August,Heavy Rain,2001-08-06 09:00:00,0,0,0,...,NaN,NaN,The remnants of Tropical Storm Barry moved nor...,None,01001,720.0,NaN,35.853419,58802.0,147.82


In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1555647 entries, 0 to 1555646
Data columns (total 28 columns):
 #   Column                  Non-Null Count    Dtype         
---  ------                  --------------    -----         
 0   BEGIN_YEARMONTH         1555647 non-null  int64         
 1   BEGIN_TIME              1555647 non-null  int64         
 2   STATE                   1555647 non-null  object        
 3   YEAR                    1555647 non-null  int64         
 4   MONTH_NAME              1555647 non-null  object        
 5   EVENT_TYPE              1555647 non-null  object        
 6   BEGIN_DATE_TIME         1555647 non-null  datetime64[ns]
 7   INJURIES_DIRECT         1555647 non-null  int64         
 8   INJURIES_INDIRECT       1555647 non-null  int64         
 9   DEATHS_DIRECT           1555647 non-null  int64         
 10  DEATHS_INDIRECT         1555647 non-null  int64         
 11  DAMAGE_PROPERTY         1555647 non-null  float64       
 12  MAGNITUDE     

In [33]:
df.head(30).T

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
BEGIN_YEARMONTH,202202,202202,202202,202202,202202,202202,202202,202202,202202,202202,...,202202,202202,202201,202201,202201,202201,202201,202201,202201,202201
BEGIN_TIME,2118,800,200,1609,0,500,100,100,200,400,...,1100,1100,1200,1200,1200,1200,1200,1200,1200,1200
STATE,NEVADA,NEVADA,NEVADA,ATLANTIC SOUTH,AMERICAN SAMOA,AMERICAN SAMOA,KANSAS,KANSAS,MASSACHUSETTS,MASSACHUSETTS,...,VERMONT,VERMONT,OHIO,OHIO,OHIO,OHIO,OHIO,OHIO,OHIO,OHIO
YEAR,2022,2022,2022,2022,2022,2022,2022,2022,2022,2022,...,2022,2022,2022,2022,2022,2022,2022,2022,2022,2022
MONTH_NAME,February,February,February,February,February,February,February,February,February,February,...,February,February,January,January,January,January,January,January,January,January
EVENT_TYPE,High Wind,Heavy Snow,Heavy Snow,Waterspout,Heavy Rain,Heavy Rain,Winter Storm,Winter Storm,Heavy Snow,Heavy Snow,...,Winter Storm,Winter Storm,Winter Weather,Winter Weather,Winter Weather,Winter Weather,Winter Weather,Winter Weather,Winter Weather,Winter Weather
BEGIN_DATE_TIME,2022-02-20 21:18:00,2022-02-21 08:00:00,2022-02-22 02:00:00,2022-02-18 16:09:00,2022-02-02 00:00:00,2022-02-12 05:00:00,2022-02-01 01:00:00,2022-02-01 01:00:00,2022-02-13 02:00:00,2022-02-13 04:00:00,...,2022-02-03 11:00:00,2022-02-03 11:00:00,2022-01-16 12:00:00,2022-01-16 12:00:00,2022-01-16 12:00:00,2022-01-16 12:00:00,2022-01-16 12:00:00,2022-01-16 12:00:00,2022-01-16 12:00:00,2022-01-16 12:00:00
INJURIES_DIRECT,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
INJURIES_INDIRECT,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
DEATHS_DIRECT,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [34]:
df.shape

(1555647, 28)

In [35]:
df.isna().sum()

BEGIN_YEARMONTH                 0
BEGIN_TIME                      0
STATE                           0
YEAR                            0
MONTH_NAME                      0
EVENT_TYPE                      0
BEGIN_DATE_TIME                 0
INJURIES_DIRECT                 0
INJURIES_INDIRECT               0
DEATHS_DIRECT                   0
DEATHS_INDIRECT                 0
DAMAGE_PROPERTY                 0
MAGNITUDE                       0
MAGNITUDE_TYPE            1038050
FLOOD_CAUSE               1439351
TOR_F_SCALE               1519607
TOR_LENGTH                      0
TOR_WIDTH                       0
BEGIN_LAT                  639054
BEGIN_LON                  639063
EPISODE_NARRATIVE          198244
EVENT_NARRATIVE            565068
ST_CT_FIPS                      0
DURATION                        0
STORM_AREA                 639063
B01001_calc_PopDensity     519894
Population                 524584
HPI                        502349
dtype: int64

In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1555647 entries, 0 to 1555646
Data columns (total 28 columns):
 #   Column                  Non-Null Count    Dtype         
---  ------                  --------------    -----         
 0   BEGIN_YEARMONTH         1555647 non-null  int64         
 1   BEGIN_TIME              1555647 non-null  int64         
 2   STATE                   1555647 non-null  object        
 3   YEAR                    1555647 non-null  int64         
 4   MONTH_NAME              1555647 non-null  object        
 5   EVENT_TYPE              1555647 non-null  object        
 6   BEGIN_DATE_TIME         1555647 non-null  datetime64[ns]
 7   INJURIES_DIRECT         1555647 non-null  int64         
 8   INJURIES_INDIRECT       1555647 non-null  int64         
 9   DEATHS_DIRECT           1555647 non-null  int64         
 10  DEATHS_INDIRECT         1555647 non-null  int64         
 11  DAMAGE_PROPERTY         1555647 non-null  float64       
 12  MAGNITUDE     

In [37]:
df.head(20).T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
BEGIN_YEARMONTH,202202,202202,202202,202202,202202,202202,202202,202202,202202,202202,202202,202202,202202,202202,202202,202202,202202,202202,202202,202202
BEGIN_TIME,2118,800,200,1609,0,500,100,100,200,400,400,300,500,2026,900,900,900,900,900,900
STATE,NEVADA,NEVADA,NEVADA,ATLANTIC SOUTH,AMERICAN SAMOA,AMERICAN SAMOA,KANSAS,KANSAS,MASSACHUSETTS,MASSACHUSETTS,MASSACHUSETTS,RHODE ISLAND,FLORIDA,MONTANA,NEW YORK,NEW YORK,NEW YORK,NEW YORK,NEW YORK,NEW YORK
YEAR,2022,2022,2022,2022,2022,2022,2022,2022,2022,2022,2022,2022,2022,2022,2022,2022,2022,2022,2022,2022
MONTH_NAME,February,February,February,February,February,February,February,February,February,February,February,February,February,February,February,February,February,February,February,February
EVENT_TYPE,High Wind,Heavy Snow,Heavy Snow,Waterspout,Heavy Rain,Heavy Rain,Winter Storm,Winter Storm,Heavy Snow,Heavy Snow,Heavy Snow,Heavy Snow,Dense Fog,Winter Storm,Winter Storm,Winter Storm,Winter Storm,Winter Storm,Winter Storm,Winter Storm
BEGIN_DATE_TIME,2022-02-20 21:18:00,2022-02-21 08:00:00,2022-02-22 02:00:00,2022-02-18 16:09:00,2022-02-02 00:00:00,2022-02-12 05:00:00,2022-02-01 01:00:00,2022-02-01 01:00:00,2022-02-13 02:00:00,2022-02-13 04:00:00,2022-02-13 04:00:00,2022-02-13 03:00:00,2022-02-25 05:00:00,2022-02-01 20:26:00,2022-02-03 09:00:00,2022-02-03 09:00:00,2022-02-03 09:00:00,2022-02-03 09:00:00,2022-02-03 09:00:00,2022-02-03 09:00:00
INJURIES_DIRECT,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
INJURIES_INDIRECT,0,0,0,0,0,0,0,0,0,0,0,0,30,0,0,0,0,0,0,0
DEATHS_DIRECT,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [38]:
df.HPI

0           341.69
1              NaN
2          1183.74
3              NaN
4              NaN
            ...   
1555642        NaN
1555643        NaN
1555644        NaN
1555645        NaN
1555646        NaN
Name: HPI, Length: 1555647, dtype: object

In [39]:
# code to replace "." so that we can ultimately turn this into a number

new_list = []
for i in df['HPI']:
    a_string = str(i)
    #a_string = df['HPI'][i]
    new_string = a_string.translate(str.maketrans('', '', string.punctuation))
    #print(a_string)
    #df['new_HPI'][i] = new_string
    #print(df['new_HPI'][i])
    new_list.append(new_string)
#    print(df['HPI'][i])    
#     df['new_HPI'][i] = new_string
#     print(df['new_HPI'][i])
#print(new_list)

In [40]:
df.HPI.notnull().value_counts()

True     1053298
False     502349
Name: HPI, dtype: int64

In [41]:
df.HPI.value_counts()

100        4397
.          1769
910.52      458
1109.35     436
671.23      350
           ... 
625.34        1
273.6         1
1332.85       1
367.36        1
115.03        1
Name: HPI, Length: 31059, dtype: int64

In [42]:
df['newHPI'] = pd.DataFrame(new_list)
#df['newHPI'] = df['newHPI'].replace('nan', 'NaN')
#df['newHPI'] = df['newHPI'].replace('', 'NaN')

In [43]:
df['newHPI']

0           34169
1             nan
2          118374
3             nan
4             nan
            ...  
1555642       nan
1555643       nan
1555644       nan
1555645       nan
1555646       nan
Name: newHPI, Length: 1555647, dtype: object

In [44]:
df[df['newHPI'] == "nan"]

,BEGIN_YEARMONTH,BEGIN_TIME,STATE,YEAR,MONTH_NAME,EVENT_TYPE,BEGIN_DATE_TIME,INJURIES_DIRECT,INJURIES_INDIRECT,DEATHS_DIRECT,...,BEGIN_LON,EPISODE_NARRATIVE,EVENT_NARRATIVE,ST_CT_FIPS,DURATION,STORM_AREA,B01001_calc_PopDensity,Population,HPI,newHPI
1,202202,800,NEVADA,2022,February,Heavy Snow,2022-02-21 08:00:00,0,0,0,...,NaN,A low centered over northern and central Nevad...,Thirteen inches fell at station (BCSN2) Big Cr...,32037,1560.0,NaN,NaN,NaN,NaN,nan
3,202202,1609,ATLANTIC SOUTH,2022,February,Waterspout,2022-02-18 16:09:00,0,0,0,...,-81.1700,Pre-frontal showers and thunderstorms moved so...,A brief waterspout was observed offshore of So...,87452,0.0,0.0000,NaN,NaN,NaN,nan
4,202202,0,AMERICAN SAMOA,2022,February,Heavy Rain,2022-02-02 00:00:00,0,0,0,...,-170.7157,A surface trough over the Islands held the po...,"Over a 24-hour period, WSO Pago Pago recorded ...",97002,1440.0,0.0063,NaN,NaN,NaN,nan
5,202202,500,AMERICAN SAMOA,2022,February,Heavy Rain,2022-02-12 05:00:00,0,0,0,...,-170.7856,A surface trough that remained anchored over t...,EOC reported a landslide near Agugulu enroute ...,97002,1020.0,0.0053,NaN,NaN,NaN,nan
6,202202,100,KANSAS,2022,February,Winter Storm,2022-02-01 01:00:00,0,0,0,...,NaN,Snow developed during the overnight hours on F...,KVOE radio station measured 5 inches of snow i...,20054,600.0,NaN,NaN,NaN,NaN,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1555642,200112,1200,PUERTO RICO,2001,December,Flood,2001-12-21 12:00:00,0,0,0,...,NaN,A cold front located north of Puerto Rico in c...,None,99001,240.0,NaN,NaN,NaN,NaN,nan
1555643,200112,2150,ARKANSAS,2001,December,Flash Flood,2001-12-16 21:50:00,0,0,0,...,NaN,"Heavy rains, amounting to between 2 and 4 inch...",None,05109,310.0,NaN,6.947737,10180.0,NaN,nan
1555644,200112,700,PUERTO RICO,2001,December,Flash Flood,2001-12-22 07:00:00,0,0,0,...,NaN,A cold front became stationary north of Puerto...,None,99001,900.0,NaN,NaN,NaN,NaN,nan
1555645,200112,700,PUERTO RICO,2001,December,Flash Flood,2001-12-22 07:00:00,0,0,0,...,NaN,A cold front became stationary north of Puerto...,None,99004,900.0,NaN,NaN,NaN,NaN,nan


In [45]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1555647 entries, 0 to 1555646
Data columns (total 29 columns):
 #   Column                  Non-Null Count    Dtype         
---  ------                  --------------    -----         
 0   BEGIN_YEARMONTH         1555647 non-null  int64         
 1   BEGIN_TIME              1555647 non-null  int64         
 2   STATE                   1555647 non-null  object        
 3   YEAR                    1555647 non-null  int64         
 4   MONTH_NAME              1555647 non-null  object        
 5   EVENT_TYPE              1555647 non-null  object        
 6   BEGIN_DATE_TIME         1555647 non-null  datetime64[ns]
 7   INJURIES_DIRECT         1555647 non-null  int64         
 8   INJURIES_INDIRECT       1555647 non-null  int64         
 9   DEATHS_DIRECT           1555647 non-null  int64         
 10  DEATHS_INDIRECT         1555647 non-null  int64         
 11  DAMAGE_PROPERTY         1555647 non-null  float64       
 12  MAGNITUDE     

In [46]:
df.newHPI.value_counts()

nan       502349
100         4397
            1769
91052        458
110935       436
           ...  
49172          1
37267          1
60753          1
70155          1
11503          1
Name: newHPI, Length: 31000, dtype: int64

In [47]:
df.newHPI.notnull().value_counts()

True    1555647
Name: newHPI, dtype: int64

In [48]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1555647 entries, 0 to 1555646
Data columns (total 29 columns):
 #   Column                  Non-Null Count    Dtype         
---  ------                  --------------    -----         
 0   BEGIN_YEARMONTH         1555647 non-null  int64         
 1   BEGIN_TIME              1555647 non-null  int64         
 2   STATE                   1555647 non-null  object        
 3   YEAR                    1555647 non-null  int64         
 4   MONTH_NAME              1555647 non-null  object        
 5   EVENT_TYPE              1555647 non-null  object        
 6   BEGIN_DATE_TIME         1555647 non-null  datetime64[ns]
 7   INJURIES_DIRECT         1555647 non-null  int64         
 8   INJURIES_INDIRECT       1555647 non-null  int64         
 9   DEATHS_DIRECT           1555647 non-null  int64         
 10  DEATHS_INDIRECT         1555647 non-null  int64         
 11  DAMAGE_PROPERTY         1555647 non-null  float64       
 12  MAGNITUDE     

In [49]:
df[df['ST_CT_FIPS'] == "01001"]

,BEGIN_YEARMONTH,BEGIN_TIME,STATE,YEAR,MONTH_NAME,EVENT_TYPE,BEGIN_DATE_TIME,INJURIES_DIRECT,INJURIES_INDIRECT,DEATHS_DIRECT,...,BEGIN_LON,EPISODE_NARRATIVE,EVENT_NARRATIVE,ST_CT_FIPS,DURATION,STORM_AREA,B01001_calc_PopDensity,Population,HPI,newHPI
1874,202206,1500,ALABAMA,2022,June,Heat,2022-06-14 15:00:00,0,0,0,...,NaN,Heat index values reached 105-107 degrees in a...,The heat index reached 105 degrees for one to ...,01001,120.0,NaN,35.853419,58802.0,238.01,23801
2486,202210,200,ALABAMA,2022,October,Frost/Freeze,2022-10-19 02:00:00,0,0,0,...,NaN,Temperatures dropped into the 26 to 32 degree ...,Temperatures dropped as low as 29 degrees.,01001,240.0,NaN,35.853419,58802.0,238.01,23801
2558,202210,2300,ALABAMA,2022,October,Frost/Freeze,2022-10-19 23:00:00,0,0,0,...,NaN,Temperatures dropped into the 26 to 32 degree ...,The temperature dropped to as low as 27 degrees.,01001,480.0,NaN,35.853419,58802.0,238.01,23801
2565,202210,500,ALABAMA,2022,October,Frost/Freeze,2022-10-18 05:00:00,0,0,0,...,NaN,Temperatures dropped into the 29 to 32 degree ...,Temperatures dropped to as low as 30 degrees d...,01001,120.0,NaN,35.853419,58802.0,238.01,23801
6282,202206,1100,ALABAMA,2022,June,Heat,2022-06-16 11:00:00,0,0,0,...,NaN,Heat index values reached 105 degrees or highe...,The heat index reached and exceeded 105 degree...,01001,420.0,NaN,35.853419,58802.0,238.01,23801
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1534818,200106,1330,ALABAMA,2001,June,Thunderstorm Wind,2001-06-04 13:30:00,0,0,0,...,-86.48333,None,A few trees were knocked down near Marbury.,01001,0.0,0.00,35.853419,58802.0,147.82,14782
1534819,200106,1410,ALABAMA,2001,June,Thunderstorm Wind,2001-06-04 14:10:00,0,0,0,...,-86.63333,None,Several trees and power lines were blown down ...,01001,15.0,0.00,35.853419,58802.0,147.82,14782
1536339,200106,1630,ALABAMA,2001,June,Thunderstorm Wind,2001-06-05 16:30:00,0,0,0,...,-86.58333,None,Trees were blown down near Booth and in the Ol...,01001,0.0,0.00,35.853419,58802.0,147.82,14782
1548495,200108,900,ALABAMA,2001,August,Heavy Rain,2001-08-06 09:00:00,0,0,0,...,NaN,The remnants of Tropical Storm Barry moved nor...,None,01001,720.0,NaN,35.853419,58802.0,147.82,14782


# START HERE

In [50]:
#df = pd.read_parquet("../Data/all_storm_data9.pqt")
#df = pd.read_parquet("../Data/all_storm_data9.pqt", index_col=[0])

In [51]:
df.head()

,BEGIN_YEARMONTH,BEGIN_TIME,STATE,YEAR,MONTH_NAME,EVENT_TYPE,BEGIN_DATE_TIME,INJURIES_DIRECT,INJURIES_INDIRECT,DEATHS_DIRECT,...,BEGIN_LON,EPISODE_NARRATIVE,EVENT_NARRATIVE,ST_CT_FIPS,DURATION,STORM_AREA,B01001_calc_PopDensity,Population,HPI,newHPI
0,202202,2118,NEVADA,2022,February,High Wind,2022-02-20 21:18:00,0,0,0,...,NaN,Strong winds increased ahead of an approaching...,"Station (UP994) 3.1 SE West Wendover, Elevatio...",32033,60.0,NaN,0.423622,9078.0,341.69,34169
1,202202,800,NEVADA,2022,February,Heavy Snow,2022-02-21 08:00:00,0,0,0,...,NaN,A low centered over northern and central Nevad...,Thirteen inches fell at station (BCSN2) Big Cr...,32037,1560.0,NaN,NaN,NaN,NaN,nan
2,202202,200,NEVADA,2022,February,Heavy Snow,2022-02-22 02:00:00,0,0,0,...,NaN,A low centered over northern and central Nevad...,Fifteen inches fell at station (TJMN2) Toe Jam...,32031,420.0,NaN,27.602298,486489.0,1183.74,118374
3,202202,1609,ATLANTIC SOUTH,2022,February,Waterspout,2022-02-18 16:09:00,0,0,0,...,-81.1700,Pre-frontal showers and thunderstorms moved so...,A brief waterspout was observed offshore of So...,87452,0.0,0.0000,NaN,NaN,NaN,nan
4,202202,0,AMERICAN SAMOA,2022,February,Heavy Rain,2022-02-02 00:00:00,0,0,0,...,-170.7157,A surface trough over the Islands held the po...,"Over a 24-hour period, WSO Pago Pago recorded ...",97002,1440.0,0.0063,NaN,NaN,NaN,nan


In [52]:
#fixing the merge

In [53]:
df.head(20).T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
BEGIN_YEARMONTH,202202,202202,202202,202202,202202,202202,202202,202202,202202,202202,202202,202202,202202,202202,202202,202202,202202,202202,202202,202202
BEGIN_TIME,2118,800,200,1609,0,500,100,100,200,400,400,300,500,2026,900,900,900,900,900,900
STATE,NEVADA,NEVADA,NEVADA,ATLANTIC SOUTH,AMERICAN SAMOA,AMERICAN SAMOA,KANSAS,KANSAS,MASSACHUSETTS,MASSACHUSETTS,MASSACHUSETTS,RHODE ISLAND,FLORIDA,MONTANA,NEW YORK,NEW YORK,NEW YORK,NEW YORK,NEW YORK,NEW YORK
YEAR,2022,2022,2022,2022,2022,2022,2022,2022,2022,2022,2022,2022,2022,2022,2022,2022,2022,2022,2022,2022
MONTH_NAME,February,February,February,February,February,February,February,February,February,February,February,February,February,February,February,February,February,February,February,February
EVENT_TYPE,High Wind,Heavy Snow,Heavy Snow,Waterspout,Heavy Rain,Heavy Rain,Winter Storm,Winter Storm,Heavy Snow,Heavy Snow,Heavy Snow,Heavy Snow,Dense Fog,Winter Storm,Winter Storm,Winter Storm,Winter Storm,Winter Storm,Winter Storm,Winter Storm
BEGIN_DATE_TIME,2022-02-20 21:18:00,2022-02-21 08:00:00,2022-02-22 02:00:00,2022-02-18 16:09:00,2022-02-02 00:00:00,2022-02-12 05:00:00,2022-02-01 01:00:00,2022-02-01 01:00:00,2022-02-13 02:00:00,2022-02-13 04:00:00,2022-02-13 04:00:00,2022-02-13 03:00:00,2022-02-25 05:00:00,2022-02-01 20:26:00,2022-02-03 09:00:00,2022-02-03 09:00:00,2022-02-03 09:00:00,2022-02-03 09:00:00,2022-02-03 09:00:00,2022-02-03 09:00:00
INJURIES_DIRECT,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
INJURIES_INDIRECT,0,0,0,0,0,0,0,0,0,0,0,0,30,0,0,0,0,0,0,0
DEATHS_DIRECT,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [54]:
df.PopDensity.value_counts()

AttributeError: 'DataFrame' object has no attribute 'PopDensity'

In [ ]:
PopDen.head(20).T

In [ ]:
df

In [ ]:
df.drop(['HPI'], axis=1, inplace=True)

In [ ]:
df = df.rename(columns={'B01001_calc_PopDensity': 'PopDensity'})

In [ ]:
df.info()

In [ ]:
df = df.rename(columns={'newHPI': 'HPI'})

In [ ]:
df['HPI'] = df['HPI'].astype(int)

#### write to CSV
from pathlib import Path  
filepath = Path('/Users/gregwelliver/Desktop/springboard_files/Severe-Weather-Repo/Data/all_storm_data5.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
df.to_csv(filepath)

#### write to parquet
parquet_file = 'example_pd.parquet'

df.to_parquet(parquet_file, engine = 'pyarrow', compression = 'gzip')

logging.info('Parquet file named "%s" has been written to disk', parquet_file)

In [ ]:
#### write to parquet
from pathlib import Path  
filepath = Path('/Users/gregwelliver/Desktop/springboard_files/Severe-Weather-Repo/Data/all_storm_data10.pqt')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
df.to_parquet(filepath)

resources

CZ FIPS documentation: https://www.irsa.miami.edu/_assets/pdf/Documents/fips_statecounty_code.pdf

Population density: https://covid19.census.gov/datasets/21843f238cbb46b08615fc53e19e0daf_1/explore?location=2.632620%2C0.315550%2C1.00

Home price index: https://www.fhfa.gov/DataTools/Downloads/Pages/House-Price-Index-Datasets.aspx

maybe useful: https://www.nar.realtor/research-and-statistics/housing-statistics/county-median-home-prices-and-monthly-mortgage-payment
        
land values: https://www.nass.usda.gov/Publications/Todays_Reports/reports/land0822.pdf

data that I created:
 - concatenated state and county codes for indentification
 - storm duration
 - storm area
 - county population density (pulled from other dataset)
 - land values (pulled from other dataset)

drop columns
# WORKING, MARKDOWN UNTIL FINAL
# drop unnecessary columns
df.drop(['', '', '', '',], axis=1, inplace=True)

# working
# merge the home price index data to the main dataframe
df = df.merge(HomePrice['HPI'], how = 'left',
                left_on = 'ST_CT_FIPS', right_on = HomePrice['FIPS code'])
#TopCountries.index = TopCountries.index + 1